Importing required libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.datasets import make_regression
from sklearn import preprocessing
import re
import warnings
warnings.filterwarnings("ignore")
import time
import os
import joblib
import pickle

Load Config

In [2]:
import yaml

#read yaml file
with open('../config/config.yaml') as file:
  config= yaml.safe_load(file)
  #print(config)

## Load Processed dataset

Get the Root Dir

In [3]:
#Get current working dir
cwd_path=os.getcwd()
#print(cwd_path)

#  Get the path of ROOT DIR
path_list = cwd_path.split(os.sep) #split path
ROOT_PATH=os.path.join(path_list[0],os.sep,*path_list[1:-2]) # To move two level up, remove last two str and join to create root path
#print(ROOT_PATH)

Read the clean & Merged data

In [4]:
# process Data file
Process_Data_file=os.path.join(ROOT_PATH,config['processed_data_dir'],config['processed_data_file_1'])
print(Process_Data_file)

# Read the Sales input file name
try:
    Data=pd.read_csv(Process_Data_file)
    #sales_data.head()
except FileNotFoundError:
    print("file {} does not exist".format(Process_Data_file))



c:\Nishikant\OneDrive - CriticalRiver Technologies Pvt. Ltd\Development\1-Team Projects\AIML_Projects\Retail Sales Forecasting\data\processed\Merged_data.csv


In [5]:
Data.head()

,ProductCategory,SalesInThousandDollars,MonthlyNominalGDPIndexinMillion,MonthlyRealGDPIndexinMillion,CPI,unemploymentrate,CommercialBankInterestRateonCreditCardPlans,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan,Earningsorwagesindollarsperhour,CottonMonthlyPriceUScentsperPoundlbs,...,Exports,Tempavg,DewPointavg,Humidityavg,SeaLevelPressavg,Visibilityavg,Windavg,Precipsum,Event,FederalHoliday
0,MenClothing,524.0,14421.752895,14407.053343,233.402,7.8,12.03,11.44,22.05,57.70,...,11.550,-2.096774,-9.903226,57.354839,1015.967742,14.000000,12.000000,2.443226,0,2
1,MenClothing,496.0,14389.200466,14366.176571,234.663,8.3,12.97,11.05,22.22,55.21,...,11.100,2.785714,-6.392857,54.000000,1017.071429,15.178571,12.214286,0.781071,1,1
2,MenClothing,542.0,14340.701639,14351.786822,235.067,8.7,12.97,11.05,22.22,51.50,...,11.650,5.967742,-2.903226,57.064516,1021.064516,14.258065,11.387097,1.434194,0,0
3,MenClothing,669.0,14326.815525,14351.601731,235.582,9.0,12.97,11.05,22.13,56.78,...,12.225,12.733333,2.533333,55.700000,1014.533333,14.066667,11.366667,3.971667,1,0
4,MenClothing,650.0,14345.904809,14368.123959,235.975,9.4,13.32,11.25,22.04,61.95,...,12.300,16.655172,9.827586,68.827586,1017.241379,12.275862,8.000000,4.528276,1,1


Reading file by specifying the path 

In [6]:
#os.getcwd()

In [7]:
#project_path='C:\\Users\\Riyaz Mehendi\\OneDrive - CriticalRiver Technologies Pvt. Ltd\\Desktop\\Retail_Sales_Forecast'
#processed_data='data\\processed'
#processed_file_name='Merged_Data.csv'

In [8]:
#processed_file_path=os.path.join(project_path,processed_data,processed_file_name)

In [9]:
#Test_Data=pd.read_csv(processed_file_path)

Genrating random 10 rows from Merged data 

In [10]:
Test_Data=Data.sample(n = 10)

In [11]:
Test_Data

,ProductCategory,SalesInThousandDollars,MonthlyNominalGDPIndexinMillion,MonthlyRealGDPIndexinMillion,CPI,unemploymentrate,CommercialBankInterestRateonCreditCardPlans,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan,Earningsorwagesindollarsperhour,CottonMonthlyPriceUScentsperPoundlbs,...,Exports,Tempavg,DewPointavg,Humidityavg,SeaLevelPressavg,Visibilityavg,Windavg,Precipsum,Event,FederalHoliday
213,OtherClothing,1158.00000,16208.314757,15292.349860,254.277,7.8,11.95,10.37,23.52,81.95,...,10.875,14.645161,9.129032,71.387097,1014.193548,13.451613,7.903226,2.393548,1,1
155,WomenClothing,5664.00000,17733.024464,16221.180125,258.080,5.6,11.99,10.47,24.59,68.30,...,9.450,4.633333,-1.866667,64.466667,1019.500000,13.100000,10.900000,5.038000,2,1
28,MenClothing,677.00000,15472.391289,14990.963816,248.073,9.0,12.89,11.37,23.06,165.52,...,15.020,17.384615,10.576923,71.000000,1014.307692,12.576923,9.307692,4.474615,1,1
154,WomenClothing,4525.00000,17765.337570,16243.054734,259.382,5.8,11.99,10.47,24.78,67.53,...,9.450,7.700000,-1.800000,54.066667,1017.400000,14.300000,8.600000,3.598667,0,2
77,MenClothing,716.00000,18188.642468,16521.301886,261.512,5.3,12.04,9.69,24.77,72.35,...,10.150,21.866667,14.300000,65.033333,1014.866667,13.100000,7.533333,4.022667,1,0
26,MenClothing,599.00000,15404.398506,15004.115470,245.617,9.0,13.44,10.98,22.90,229.67,...,15.280,5.838710,-3.387097,56.096774,1019.612903,14.000000,12.290323,5.072258,0,0
89,WomenClothing,2220.00000,14348.530666,14346.820106,237.172,9.5,13.32,11.25,21.96,61.39,...,10.300,19.678571,15.250000,79.178571,1011.607143,11.571429,7.357143,9.108571,1,0
225,OtherClothing,1746.60251,16884.854816,15729.959366,257.069,7.2,11.88,10.13,24.04,89.35,...,9.650,15.774194,7.741935,60.774194,1018.193548,14.774194,8.096774,0.295806,1,1
72,MenClothing,560.00000,17792.116953,16319.786314,258.376,5.7,11.99,10.47,24.88,67.35,...,9.450,-1.032258,-9.612903,53.645161,1019.387097,13.032258,11.677419,4.284839,0,2
37,MenClothing,545.00000,16073.575390,15385.847018,250.285,8.3,12.34,10.89,23.40,100.75,...,10.350,5.142857,-3.678571,55.821429,1016.714286,14.321429,10.714286,1.234643,1,1


In [12]:
Test_Data.reset_index(drop=True, inplace=True)

In [13]:
Test_Data

,ProductCategory,SalesInThousandDollars,MonthlyNominalGDPIndexinMillion,MonthlyRealGDPIndexinMillion,CPI,unemploymentrate,CommercialBankInterestRateonCreditCardPlans,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan,Earningsorwagesindollarsperhour,CottonMonthlyPriceUScentsperPoundlbs,...,Exports,Tempavg,DewPointavg,Humidityavg,SeaLevelPressavg,Visibilityavg,Windavg,Precipsum,Event,FederalHoliday
0,OtherClothing,1158.00000,16208.314757,15292.349860,254.277,7.8,11.95,10.37,23.52,81.95,...,10.875,14.645161,9.129032,71.387097,1014.193548,13.451613,7.903226,2.393548,1,1
1,WomenClothing,5664.00000,17733.024464,16221.180125,258.080,5.6,11.99,10.47,24.59,68.30,...,9.450,4.633333,-1.866667,64.466667,1019.500000,13.100000,10.900000,5.038000,2,1
2,MenClothing,677.00000,15472.391289,14990.963816,248.073,9.0,12.89,11.37,23.06,165.52,...,15.020,17.384615,10.576923,71.000000,1014.307692,12.576923,9.307692,4.474615,1,1
3,WomenClothing,4525.00000,17765.337570,16243.054734,259.382,5.8,11.99,10.47,24.78,67.53,...,9.450,7.700000,-1.800000,54.066667,1017.400000,14.300000,8.600000,3.598667,0,2
4,MenClothing,716.00000,18188.642468,16521.301886,261.512,5.3,12.04,9.69,24.77,72.35,...,10.150,21.866667,14.300000,65.033333,1014.866667,13.100000,7.533333,4.022667,1,0
5,MenClothing,599.00000,15404.398506,15004.115470,245.617,9.0,13.44,10.98,22.90,229.67,...,15.280,5.838710,-3.387097,56.096774,1019.612903,14.000000,12.290323,5.072258,0,0
6,WomenClothing,2220.00000,14348.530666,14346.820106,237.172,9.5,13.32,11.25,21.96,61.39,...,10.300,19.678571,15.250000,79.178571,1011.607143,11.571429,7.357143,9.108571,1,0
7,OtherClothing,1746.60251,16884.854816,15729.959366,257.069,7.2,11.88,10.13,24.04,89.35,...,9.650,15.774194,7.741935,60.774194,1018.193548,14.774194,8.096774,0.295806,1,1
8,MenClothing,560.00000,17792.116953,16319.786314,258.376,5.7,11.99,10.47,24.88,67.35,...,9.450,-1.032258,-9.612903,53.645161,1019.387097,13.032258,11.677419,4.284839,0,2
9,MenClothing,545.00000,16073.575390,15385.847018,250.285,8.3,12.34,10.89,23.40,100.75,...,10.350,5.142857,-3.678571,55.821429,1016.714286,14.321429,10.714286,1.234643,1,1


Creating Dataframe with only numeric columns

In [14]:
Test_Numeric = Test_Data.drop(['ProductCategory'], axis = 1)

In [15]:
Test_Numeric

,SalesInThousandDollars,MonthlyNominalGDPIndexinMillion,MonthlyRealGDPIndexinMillion,CPI,unemploymentrate,CommercialBankInterestRateonCreditCardPlans,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan,Earningsorwagesindollarsperhour,CottonMonthlyPriceUScentsperPoundlbs,Changein,...,Exports,Tempavg,DewPointavg,Humidityavg,SeaLevelPressavg,Visibilityavg,Windavg,Precipsum,Event,FederalHoliday
0,1158.00000,16208.314757,15292.349860,254.277,7.8,11.95,10.37,23.52,81.95,-2.61,...,10.875,14.645161,9.129032,71.387097,1014.193548,13.451613,7.903226,2.393548,1,1
1,5664.00000,17733.024464,16221.180125,258.080,5.6,11.99,10.47,24.59,68.30,1.14,...,9.450,4.633333,-1.866667,64.466667,1019.500000,13.100000,10.900000,5.038000,2,1
2,677.00000,15472.391289,14990.963816,248.073,9.0,12.89,11.37,23.06,165.52,-23.59,...,15.020,17.384615,10.576923,71.000000,1014.307692,12.576923,9.307692,4.474615,1,1
3,4525.00000,17765.337570,16243.054734,259.382,5.8,11.99,10.47,24.78,67.53,-3.99,...,9.450,7.700000,-1.800000,54.066667,1017.400000,14.300000,8.600000,3.598667,0,2
4,716.00000,18188.642468,16521.301886,261.512,5.3,12.04,9.69,24.77,72.35,-0.70,...,10.150,21.866667,14.300000,65.033333,1014.866667,13.100000,7.533333,4.022667,1,0
5,599.00000,15404.398506,15004.115470,245.617,9.0,13.44,10.98,22.90,229.67,7.74,...,15.280,5.838710,-3.387097,56.096774,1019.612903,14.000000,12.290323,5.072258,0,0
6,2220.00000,14348.530666,14346.820106,237.172,9.5,13.32,11.25,21.96,61.39,-0.90,...,10.300,19.678571,15.250000,79.178571,1011.607143,11.571429,7.357143,9.108571,1,0
7,1746.60251,16884.854816,15729.959366,257.069,7.2,11.88,10.13,24.04,89.35,-0.82,...,9.650,15.774194,7.741935,60.774194,1018.193548,14.774194,8.096774,0.295806,1,1
8,560.00000,17792.116953,16319.786314,258.376,5.7,11.99,10.47,24.88,67.35,-1.39,...,9.450,-1.032258,-9.612903,53.645161,1019.387097,13.032258,11.677419,4.284839,0,2
9,545.00000,16073.575390,15385.847018,250.285,8.3,12.34,10.89,23.40,100.75,-0.36,...,10.350,5.142857,-3.678571,55.821429,1016.714286,14.321429,10.714286,1.234643,1,1


Creating Independent and Target features

In [16]:
Indpendent_features= Test_Data[['MonthlyNominalGDPIndexinMillion', 'MonthlyRealGDPIndexinMillion',
       'CPI', 'unemploymentrate',
       'CommercialBankInterestRateonCreditCardPlans',
       'FinanceRateonPersonalLoansatCommercialBanks24MonthLoan',
       'Earningsorwagesindollarsperhour',
       'CottonMonthlyPriceUScentsperPoundlbs', 'Changein',
       'Averageuplandplantedmillionacres',
       'Averageuplandharvestedmillionacres', 'yieldperharvestedacre',
       'Productionin480lbnetwerightinmillionbales',
       'Millusein480lbnetwerightinmillionbales', 'Exports', 'Tempavg',
       'DewPointavg', 'Humidityavg', 'SeaLevelPressavg', 'Visibilityavg',
       'Windavg', 'Precipsum', 'Event', 'FederalHoliday']]

In [17]:
Target_features=Test_Data[['SalesInThousandDollars']]

Normalising all the features

In [18]:
#defining a normalisation function 
def normalize (x): 
    return ( (x-np.min(x))/ (max(x) - min(x)))

In [19]:
# applying normalize ( ) to all columns 
Indpendent_features = Indpendent_features.apply(normalize)

Dropping columns which were removed from train data during feature selection method 

In [20]:
Test_Data= Test_Data.drop(['Averageuplandharvestedmillionacres','Productionin480lbnetwerightinmillionbales',
                                                 'Tempavg','DewPointavg','Humidityavg', ], axis = 1)

In [21]:
Test_Data

,ProductCategory,SalesInThousandDollars,MonthlyNominalGDPIndexinMillion,MonthlyRealGDPIndexinMillion,CPI,unemploymentrate,CommercialBankInterestRateonCreditCardPlans,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan,Earningsorwagesindollarsperhour,CottonMonthlyPriceUScentsperPoundlbs,...,Averageuplandplantedmillionacres,yieldperharvestedacre,Millusein480lbnetwerightinmillionbales,Exports,SeaLevelPressavg,Visibilityavg,Windavg,Precipsum,Event,FederalHoliday
0,OtherClothing,1158.00000,16208.314757,15292.349860,254.277,7.8,11.95,10.37,23.52,81.95,...,12.121,782,3.375,10.875,1014.193548,13.451613,7.903226,2.393548,1,1
1,WomenClothing,5664.00000,17733.024464,16221.180125,258.080,5.6,11.99,10.47,24.59,68.30,...,10.818,760,3.775,9.450,1019.500000,13.100000,10.900000,5.038000,2,1
2,MenClothing,677.00000,15472.391289,14990.963816,248.073,9.0,12.89,11.37,23.06,165.52,...,10.769,805,3.760,15.020,1014.307692,12.576923,9.307692,4.474615,1,1
3,WomenClothing,4525.00000,17765.337570,16243.054734,259.382,5.8,11.99,10.47,24.78,67.53,...,10.818,783,3.775,9.450,1017.400000,14.300000,8.600000,3.598667,0,2
4,MenClothing,716.00000,18188.642468,16521.301886,261.512,5.3,12.04,9.69,24.77,72.35,...,9.399,797,3.775,10.150,1014.866667,13.100000,7.533333,4.022667,1,0
5,MenClothing,599.00000,15404.398506,15004.115470,245.617,9.0,13.44,10.98,22.90,229.67,...,10.769,814,3.565,15.280,1019.612903,14.000000,12.290323,5.072258,0,0
6,WomenClothing,2220.00000,14348.530666,14346.820106,237.172,9.5,13.32,11.25,21.96,61.39,...,8.668,796,3.470,10.300,1011.607143,11.571429,7.357143,9.108571,1,0
7,OtherClothing,1746.60251,16884.854816,15729.959366,257.069,7.2,11.88,10.13,24.04,89.35,...,10.136,777,3.480,9.650,1018.193548,14.774194,8.096774,0.295806,1,1
8,MenClothing,560.00000,17792.116953,16319.786314,258.376,5.7,11.99,10.47,24.88,67.35,...,10.845,760,3.775,9.450,1019.387097,13.032258,11.677419,4.284839,0,2
9,MenClothing,545.00000,16073.575390,15385.847018,250.285,8.3,12.34,10.89,23.40,100.75,...,14.426,754,3.475,10.350,1016.714286,14.321429,10.714286,1.234643,1,1


In [22]:
Test_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 21 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   ProductCategory                                         10 non-null     object 
 1   SalesInThousandDollars                                  10 non-null     float64
 2   MonthlyNominalGDPIndexinMillion                         10 non-null     float64
 3   MonthlyRealGDPIndexinMillion                            10 non-null     float64
 4   CPI                                                     10 non-null     float64
 5   unemploymentrate                                        10 non-null     float64
 6   CommercialBankInterestRateonCreditCardPlans             10 non-null     float64
 7   FinanceRateonPersonalLoansatCommercialBanks24MonthLoan  10 non-null     float64
 8   Earningsorwagesindollarsperhour            

 product_category needs to be converted into numeric

In [23]:
## converting text data into numerical
Test_Data.replace({'MenClothing': 0, 'WomenClothing': 1,'OtherClothing':2},inplace=True)

In [24]:
# label_encoder
#label_encoder = preprocessing.LabelEncoder()

In [25]:
#Test_Data['ProductCategory']= label_encoder.fit_transform(Test_Data['ProductCategory'])

In [26]:
Test_Data

,ProductCategory,SalesInThousandDollars,MonthlyNominalGDPIndexinMillion,MonthlyRealGDPIndexinMillion,CPI,unemploymentrate,CommercialBankInterestRateonCreditCardPlans,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan,Earningsorwagesindollarsperhour,CottonMonthlyPriceUScentsperPoundlbs,...,Averageuplandplantedmillionacres,yieldperharvestedacre,Millusein480lbnetwerightinmillionbales,Exports,SeaLevelPressavg,Visibilityavg,Windavg,Precipsum,Event,FederalHoliday
0,2,1158.00000,16208.314757,15292.349860,254.277,7.8,11.95,10.37,23.52,81.95,...,12.121,782,3.375,10.875,1014.193548,13.451613,7.903226,2.393548,1,1
1,1,5664.00000,17733.024464,16221.180125,258.080,5.6,11.99,10.47,24.59,68.30,...,10.818,760,3.775,9.450,1019.500000,13.100000,10.900000,5.038000,2,1
2,0,677.00000,15472.391289,14990.963816,248.073,9.0,12.89,11.37,23.06,165.52,...,10.769,805,3.760,15.020,1014.307692,12.576923,9.307692,4.474615,1,1
3,1,4525.00000,17765.337570,16243.054734,259.382,5.8,11.99,10.47,24.78,67.53,...,10.818,783,3.775,9.450,1017.400000,14.300000,8.600000,3.598667,0,2
4,0,716.00000,18188.642468,16521.301886,261.512,5.3,12.04,9.69,24.77,72.35,...,9.399,797,3.775,10.150,1014.866667,13.100000,7.533333,4.022667,1,0
5,0,599.00000,15404.398506,15004.115470,245.617,9.0,13.44,10.98,22.90,229.67,...,10.769,814,3.565,15.280,1019.612903,14.000000,12.290323,5.072258,0,0
6,1,2220.00000,14348.530666,14346.820106,237.172,9.5,13.32,11.25,21.96,61.39,...,8.668,796,3.470,10.300,1011.607143,11.571429,7.357143,9.108571,1,0
7,2,1746.60251,16884.854816,15729.959366,257.069,7.2,11.88,10.13,24.04,89.35,...,10.136,777,3.480,9.650,1018.193548,14.774194,8.096774,0.295806,1,1
8,0,560.00000,17792.116953,16319.786314,258.376,5.7,11.99,10.47,24.88,67.35,...,10.845,760,3.775,9.450,1019.387097,13.032258,11.677419,4.284839,0,2
9,0,545.00000,16073.575390,15385.847018,250.285,8.3,12.34,10.89,23.40,100.75,...,14.426,754,3.475,10.350,1016.714286,14.321429,10.714286,1.234643,1,1


Separating Independent features and Target feature 

In [27]:
X=Test_Data.drop(['SalesInThousandDollars'],axis = 1)

In [28]:
X

,ProductCategory,MonthlyNominalGDPIndexinMillion,MonthlyRealGDPIndexinMillion,CPI,unemploymentrate,CommercialBankInterestRateonCreditCardPlans,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan,Earningsorwagesindollarsperhour,CottonMonthlyPriceUScentsperPoundlbs,Changein,Averageuplandplantedmillionacres,yieldperharvestedacre,Millusein480lbnetwerightinmillionbales,Exports,SeaLevelPressavg,Visibilityavg,Windavg,Precipsum,Event,FederalHoliday
0,2,16208.314757,15292.349860,254.277,7.8,11.95,10.37,23.52,81.95,-2.61,12.121,782,3.375,10.875,1014.193548,13.451613,7.903226,2.393548,1,1
1,1,17733.024464,16221.180125,258.080,5.6,11.99,10.47,24.59,68.30,1.14,10.818,760,3.775,9.450,1019.500000,13.100000,10.900000,5.038000,2,1
2,0,15472.391289,14990.963816,248.073,9.0,12.89,11.37,23.06,165.52,-23.59,10.769,805,3.760,15.020,1014.307692,12.576923,9.307692,4.474615,1,1
3,1,17765.337570,16243.054734,259.382,5.8,11.99,10.47,24.78,67.53,-3.99,10.818,783,3.775,9.450,1017.400000,14.300000,8.600000,3.598667,0,2
4,0,18188.642468,16521.301886,261.512,5.3,12.04,9.69,24.77,72.35,-0.70,9.399,797,3.775,10.150,1014.866667,13.100000,7.533333,4.022667,1,0
5,0,15404.398506,15004.115470,245.617,9.0,13.44,10.98,22.90,229.67,7.74,10.769,814,3.565,15.280,1019.612903,14.000000,12.290323,5.072258,0,0
6,1,14348.530666,14346.820106,237.172,9.5,13.32,11.25,21.96,61.39,-0.90,8.668,796,3.470,10.300,1011.607143,11.571429,7.357143,9.108571,1,0
7,2,16884.854816,15729.959366,257.069,7.2,11.88,10.13,24.04,89.35,-0.82,10.136,777,3.480,9.650,1018.193548,14.774194,8.096774,0.295806,1,1
8,0,17792.116953,16319.786314,258.376,5.7,11.99,10.47,24.88,67.35,-1.39,10.845,760,3.775,9.450,1019.387097,13.032258,11.677419,4.284839,0,2
9,0,16073.575390,15385.847018,250.285,8.3,12.34,10.89,23.40,100.75,-0.36,14.426,754,3.475,10.350,1016.714286,14.321429,10.714286,1.234643,1,1


In [29]:
y=Test_Data['SalesInThousandDollars']

In [30]:
y

0    1158.00000
1    5664.00000
2     677.00000
3    4525.00000
4     716.00000
5     599.00000
6    2220.00000
7    1746.60251
8     560.00000
9     545.00000
Name: SalesInThousandDollars, dtype: float64

## Load the model

In [31]:
# Prediction model filename
prediction_model_file=os.path.join(ROOT_PATH,config['models_dir'],config['prediction_model'])
#print(prediction_model_file)

In [32]:
## load the model
prediction_model = pickle.load(open(prediction_model_file, 'rb'))

In [33]:
## lets predict the labels for x data

y_pred=prediction_model.predict(X)
y_pred

array([1277.3916 , 4748.2153 ,  681.0755 , 3544.644  ,  734.9953 ,
        700.4935 , 2246.8037 , 1672.2162 ,  559.7428 ,  581.11957],
      dtype=float32)

In [34]:
print(mean_absolute_percentage_error(y,y_pred)*100)

8.048245477251744
